## Connecting to the Database

In [1]:
pip install psycopg2-binary;

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2

In [3]:
# Put the password in 
PGHOST = 'tripdatabase.cmaaautpgbsf.us-east-2.rds.amazonaws.com'
PGDATABASE = ''
PGUSER = 'postgres'
PGPASSWORD = ''

In [4]:
try:   
    # Set up a connection to the postgres server.    
    conn = psycopg2.connect(user = PGUSER,
                            port = "5432",
                            password = PGPASSWORD,
                            host = PGHOST,
                            database = PGDATABASE)
    # Create a cursor object
    cursor = conn.cursor()   
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("Connection Success:", record,"\n")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

Connection Success: ('PostgreSQL 12.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 



## Populating the Database

In [5]:
pip install s3fs;

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import s3fs
import os
from io import StringIO
import timeit

In [7]:
qtable = """
        CREATE TABLE IF NOT EXISTS trips(
                starttime TIMESTAMP,
                endtime TIMESTAMP,
                tripduration INTEGER, 
                startID SMALLINT,
                endID SMALLINT
        );
        """
cursor.execute("rollback;")
cursor.execute(qtable)
conn.commit()

In [8]:
ACCESS_KEY_ID = ''
ACCESS_SECRET_KEY = ''
bucket = "s3://williams-citibike/TripData/"
fs = s3fs.S3FileSystem(anon=False, key = ACCESS_KEY_ID, secret= ACCESS_SECRET_KEY)

In [17]:
def populate_database(datafile: str) -> None:
    cols = ["starttime","stoptime","tripduration","start station id", "end station id"]
    datastream = StringIO()
    
    with fs.open("s3://"+datafile, 'r') as f:
        data = pd.read_csv(f, usecols=cols, parse_dates = ['starttime','stoptime'])
        data = data[cols]
        data.rename(columns={"stoptime":"endtime", "start station id":"startID", "end station id":"endID"}, inplace=True)
        
        data.to_csv(datastream, index=False, header = False)
        datastream.seek(0)

        cursor.copy_from(datastream,'trips',sep=',')
        conn.commit()
    
    datastream.close()
    print(f"finished with {datafile}")

In [18]:
filenames = fs.ls("s3://williams-citibike/TripData/")[1:]
populate_database(filenames[0])

finished with williams-citibike/TripData/2014-01 - Citi Bike trip data.csv


In [11]:
for file in filenames:
    populate_database(file)

KeyboardInterrupt: 

In [ ]:
if(conn):
    cursor.close()
    conn.close()
    print("PostgreSQL connection is closed")